# Music Performer (v.1.0)

### This is a work in progress so please check back for updates and improvements.

***

### Based on the code/repo of @lucidrains of GitHub:

https://github.com/lucidrains/performer-pytorch

***

Project Los Angeles

Tegridy Code 2021

***

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools
!pip install performer-pytorch

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" ./apex

In [ ]:
!sh setup.sh

In [ ]:
%cd /content/tegridy-tools/tegridy-tools
import TMIDI
%cd /content/

import os

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

from performer_pytorch import PerformerLM
from performer_pytorch.autoregressive_wrapper import AutoregressiveWrapper
from apex import amp

import random
import tqdm
import gzip
import numpy as np
import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler

In [ ]:
# constants

NUM_BATCHES = int(1e5)
BATCH_SIZE = 12
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 3e-4
VALIDATE_EVERY  = 100
GENERATE_EVERY  = 500
GENERATE_LENGTH = 2048
SEQ_LEN = 4096

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

def decode_token(token):
    return str(chr(max(32, token)))

def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))

# instantiate model

model = PerformerLM(
    num_tokens = 256,
    dim = 512,
    depth = 6,
    max_seq_len = SEQ_LEN,
    heads = 8,
    causal = True,
    reversible = True,
    nb_features = 256,
    use_scalenorm = True,
    local_attn_heads = (8, 8, 8, 6, 4, 2)
)

model = AutoregressiveWrapper(model)
model.cuda()

In [ ]:
# prepare enwik8 data

with open('/content/Music-Reformer_INT_Dataset.txt') as file:
    X = file.read()
    Y = []
    for x in X.split('\n'):
      Y.append(int(x))
    trX, vaX = np.split(Y, [int(1000000)])
    data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len - 1, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = GradScaler()

# training

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        with autocast():
            loss = model(next(train_loader), return_loss = True)
        scaler.scale(loss).backward()

    print(f'training loss: {loss.item()}')

    scaler.unscale_(optim)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    scaler.step(optim)
    scaler.update()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            loss = model(next(val_loader), return_loss = True)
            print(f'validation loss: {loss.item()}')

    if i % GENERATE_EVERY == 0 and i != 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        prime = decode_tokens(inp)
        print(f'%s \n\n %s', (prime, '*' * 100))

        sample = model.generate(inp, GENERATE_LENGTH)
        output_str = decode_tokens(sample)
        print(output_str)

In [ ]:
model.eval()
inp = random.choice(val_dataset)[:-1]
prime = decode_tokens(inp)
print(f'%s \n\n %s', (prime, '*' * 100))

sample = model.generate(inp, GENERATE_LENGTH, temperature=0.7)
output_str = decode_tokens(sample)
print(output_str)

In [ ]:
#@title Convert generated output to MIDI.
# Run the cells below to convert generated output to MIDI.
# If you getting errors/halts, regenerate the output again.
# Model must be sufficiently trained. Rec. 0.90+ accuracy for the output to make sense and pass error control.

#TXT = TMIDI.Tegridy_INT_String_to_TXT_Converter(input, line_by_line_input=False)
SONG = TMIDI.Tegridy_Optimus_TXT_to_Notes_Converter(output_str, has_MIDI_channels=False, char_encoding_offset=33, simulate_velocity=False, dataset_MIDI_events_time_denominator=10, line_by_line_dataset=False, has_velocities=True)
stats = TMIDI.Tegridy_SONG_to_MIDI_Converter(SONG[0], output_file_name='/content/Music-Reformer_MIDI')
print(stats)